In [21]:
from bs4 import BeautifulSoup
import re, time
import os
import urllib.request
from io import BytesIO
from PIL import Image
from IPython.display import Image as IMG, display 

#import matplotlib.pyplot as plt
import hashlib
import time

class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"    
    
def loadImgFromUrl(link):
    print("load img from url "+str(link))
    file = BytesIO(urllib.request.urlopen(link).read())
    img = Image.open(file)
    #showImg(img)
    return img
    
    
def getSubImgsFromSite(link):
    print("get sub imgs from site "+str(link))
    opener = AppURLopener()
    response = opener.open(link).read()
    soup = BeautifulSoup(response, "lxml")
    
    rawList=soup.findAll("section", {"class": "media"})
    #print("Raw list is"+str(rawList))
    urls = [a.find_all('a', href=True) for a in rawList]
    #print(urls)
    real_imgs=[]
    
    try:
        for raw_url in urls[0]:
            #print(raw_url.get('href'))
            l=raw_url.get('href')
            if l and "youtube" not in l and "https" in l:
                real_imgs.append(loadImgFromUrl(l))
    except IndexError as e:
        print("OOPS: IndexError occured")
    return real_imgs
    
def getDataFromLink(link):
    opener = AppURLopener()
    response = opener.open(link).read()
    soup = BeautifulSoup(response, "lxml")
    
    rawList=soup.findAll("li", {"class": "residental"})
    #print(rawList)
    thumbs = [a.find("li") for a in rawList]
    #print(thumbs)
    
    facts = [a.find("a", {"class": "facts"}) for a in rawList]
    
    links = [a.get('href').replace("..","https://www.etuovi.com") for a in facts]
    print(links)
    
    imgs=[]
    for l in links:
        print("Link is "+str(l))
        if l and "https" in l:
            imgs.append(getSubImgsFromSite(l))
        else:
            imgs.append([])
    #print(imgs)
    
    cities = [str(a.find("div", {"class": "address"}).span).replace("<span>","").replace("</span>","") for a in facts]
    #print(cities)
    
    addresses = [str(a.find("div", {"class": "address"}).strong).replace("<strong>","").replace("</strong>","") for a in facts]
    #print(addresses)
    
    house_type = [str(a.find("div", {"class": "type"}).label).replace("<label>","").replace("</label>","") for a in facts]
    #print(house_type)
    
    house_spec = [str(a.find("div", {"class": "type"}).span).replace("<span>","").replace("</span>","") for a in facts]
    #print(house_spec)
    
    sizes = [str(a.find("div", {"class": "size"}).span).replace("<span>","").replace("</span>","") for a in facts]
    #print(sizes)
    
    prices = [str(a.find("div", {"class": "price"}).span).replace("<span>","").replace("</span>","") for a in facts]
    #print(prices)
    
    years = [str(a.find("div", {"class": "year"}).span).replace("<span>","").replace("</span>","") for a in facts]
    #print(years)
    
    #Transpose data
    data=[]
    for i1,i2,i3,i4,i5,i6,i7,i8 in zip(links,cities,addresses,house_type,house_spec,sizes,prices,years):
        data.append([i1,i2,i3,i4,i5,i6,i7,i8])
    return [data,links, imgs]


import csv
def writeToCsv(file,data):
    file=os.path.expanduser(file)
    with open(file,'a') as f:
        wr = csv.writer(f, dialect='excel')
        wr.writerows(data)

def saveImgs(links,imgs,path):
    path=os.path.expanduser(path)
    for l,ims in zip(links,imgs):
        hsh=str(hashlib.sha224(l.encode('utf-8')).hexdigest())
        count=0
        for i in ims:
            count=count+1
            print("Savimg "+path+"/"+hsh+str(count).zfill(2)+".jpg")
            i.save(path+"/"+hsh+str(count).zfill(2)+".jpg", "JPEG")
test=0      
if test==1:    
    data,links,imgs=getDataFromLink('https://www.etuovi.com/myytavat-asunnot')

    dir=os.path.expanduser("~/DataMiner/house data/images")
    if not os.path.exists(dir):
        os.makedirs(dir)
    saveImgs(links,imgs,dir)

    #for d in data: print(d)
    writeToCsv("~/DataMiner/house data/test data houses.csv", data)
    print("Done")

In [2]:
'''def showImg(img):
    %matplotlib inline
    if img != None:
        imgplot = plt.imshow(img)'''


In [11]:
#writeToCsv("~/DataMiner/house data/test data houses.csv", data)

In [ ]:
import datetime

#import progressbar
dir_name=os.path.expanduser("~/DataMiner/house data/")
imgdir_name=os.path.expanduser("~/DataMiner/house data/images")
if not os.path.exists(imgdir_name):
    os.makedirs(imgdir_name)
time=str(datetime.datetime.now().date())
filename=time + " etuovi.csv"
f=open(dir_name+filename,'a')
f.close()

raw_link='https://www.etuovi.com/myytavat-asunnot/?page='
last_page=7000
        
#bar = progressbar.ProgressBar(maxval=last_page, \
#    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
#bar.start()
failed=0
for i in range(1,last_page):
    #Try getting, try saving and try saving seperately?
    try:
        print("Processing page: "+raw_link+str(i))
        data,links,imgs=getDataFromLink(raw_link+str(i))
        print("Saving")
        saveImgs(links,imgs,imgdir_name)
        writeToCsv(dir_name+filename, data)
    except:
        print("Round failed ",i)
        failed=failed+1
    finally:
        if i%2==0:
            print("Progress "+str(i/last_page)+" successfull "+str(i-failed)+" failed "+str(failed))
print("Finished")
print("Failed ", failed)
print("Succeeded ", last_page-failed)
#bar.finish()

Processing page: https://www.etuovi.com/myytavat-asunnot/?page=1


/usr/lib/python3/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods


['https://www.etuovi.com/kohde/9996605?sc=LRnomun_nodistr&pos=1', 'https://www.etuovi.com/kohde/9671647?sc=LRnomun_nodistr&pos=2', 'https://www.etuovi.com/kohde/8157095?sc=LRnomun_nodistr&pos=3', 'https://www.etuovi.com/kohde/8103841?sc=LRnomun_nodistr&pos=4', 'https://www.etuovi.com/kohde/8109487?sc=LRnomun_nodistr&pos=5', 'https://www.etuovi.com/kohde/8134837?sc=LRnomun_nodistr&pos=6', 'https://www.etuovi.com/kohde/8162952?sc=LRnomun_nodistr&pos=7', 'https://www.etuovi.com/kohde/8191550?sc=LRnomun_nodistr&pos=8', 'https://www.etuovi.com/kohde/7228232?sc=LRnomun_nodistr&pos=9', 'https://www.etuovi.com/kohde/7253360?sc=LRnomun_nodistr&pos=10']
Link is https://www.etuovi.com/kohde/9996605?sc=LRnomun_nodistr&pos=1
get sub imgs from site https://www.etuovi.com/kohde/9996605?sc=LRnomun_nodistr&pos=1


/usr/lib/python3/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods


load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/595/1054595/0e76b9541118d347871c0e0e51d244c4/e102320f61471fcc58db2e6f8980f6f4/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/595/1054595/0e76b9541118d347871c0e0e51d244c4/ccbc049ed63bc1736ddc88618e620d7d/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/595/1054595/0e76b9541118d347871c0e0e51d244c4/c36a0f4d86d7164271d9dd050728e551/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/595/1054595/0e76b9541118d347871c0e0e51d244c4/f28d3ebd701c21061c6fe97e7e1b8d45/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/595/1054595/0e76b9541118d347871c0e0e51d244c4/9e6fd4c0628db498e68729ce52ec45b2/ORIGINAL.jpeg
load img f

...

load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/upload/458/458020/nopid/original-1989154808136826153.jpg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/upload/458/458020/nopid/original-5141192112382022834.jpg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/upload/458/458020/nopid/original-7933961873322363216.jpg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/upload/458/458020/nopid/original-8632662049845391227.jpg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/upload/458/458020/nopid/original-4192234774436896815.jpg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/upload/458/458020/nopid/original-904602354305714403.jpg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x43

Savimg /home/m/DataMiner/house data/images/688751474df18b51027f63baca9eeb4614aee5c03778c1c081b5dae908.jpg
Savimg /home/m/DataMiner/house data/images/688751474df18b51027f63baca9eeb4614aee5c03778c1c081b5dae909.jpg
Savimg /home/m/DataMiner/house data/images/688751474df18b51027f63baca9eeb4614aee5c03778c1c081b5dae910.jpg
Savimg /home/m/DataMiner/house data/images/688751474df18b51027f63baca9eeb4614aee5c03778c1c081b5dae911.jpg
Savimg /home/m/DataMiner/house data/images/688751474df18b51027f63baca9eeb4614aee5c03778c1c081b5dae912.jpg
Savimg /home/m/DataMiner/house data/images/688751474df18b51027f63baca9eeb4614aee5c03778c1c081b5dae913.jpg
Savimg /home/m/DataMiner/house data/images/688751474df18b51027f63baca9eeb4614aee5c03778c1c081b5dae914.jpg
Savimg /home/m/DataMiner/house data/images/688751474df18b51027f63baca9eeb4614aee5c03778c1c081b5dae915.jpg
Savimg /home/m/DataMiner/house data/images/688751474df18b51027f63baca9eeb4614aee5c03778c1c081b5dae916.jpg
Savimg /home/m/DataMiner/house data/images/688

load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/120/1054120/0e76b9541118d347871c0e0e51d244c4/0af9e9d25ace255a910e108cda62aff2/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/120/1054120/0e76b9541118d347871c0e0e51d244c4/d4fce217f8ff782ef27e9ae860f49d98/ORIGINAL.png
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/120/1054120/0e76b9541118d347871c0e0e51d244c4/25d4a0e6fc283cf80ac9a5268a95917c/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/120/1054120/0e76b9541118d347871c0e0e51d244c4/acea268255984952b37714d2e7499f39/ORIGINAL.jpeg
Link is https://www.etuovi.com/kohde/1244188?sc=LRnomun_nodistr&pos=472
get sub imgs from site https://www.etuovi.com/kohde/1244188?sc=LRnomun_nodistr&pos=472
load img from url https://d3ls91xgksobn.cloudfront

load img from url https://d3ls91xgksobn.cloudfront.net:443/978x652,fit/etuovimedia/images/property/import/118/1054118/0e76b9541118d347871c0e0e51d244c4/44ad0c3373805ec392ace8b8c85ebc6d/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/978x652,fit/etuovimedia/images/property/import/118/1054118/0e76b9541118d347871c0e0e51d244c4/6f49aa538f57d7127cae46a3f9bd2627/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/978x652,fit/etuovimedia/images/property/import/118/1054118/0e76b9541118d347871c0e0e51d244c4/b0d5cd5c290ed31d892aa67a5017f6f0/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/978x652,fit/etuovimedia/images/property/import/118/1054118/0e76b9541118d347871c0e0e51d244c4/32416a8e680cb9b4442c1ede9e68a99d/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/978x652,fit/etuovimedia/images/property/import/118/1054118/0e76b9541118d347871c0e0e51d244c4/1685d57c58f52048487f8e6de1aea16c/ORIGINAL.jpeg
load img f

load img from url https://d3ls91xgksobn.cloudfront.net:443/978x652,fit/etuovimedia/images/property/import/111/1054111/0e76b9541118d347871c0e0e51d244c4/2f12dcc950f7e23921b0c0b54b47d0f7/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/images/ogroup/promotional/promotionalimage23_7.jpg
Link is https://www.etuovi.com/kohde/641568?sc=LRnomun_nodistr&pos=482
get sub imgs from site https://www.etuovi.com/kohde/641568?sc=LRnomun_nodistr&pos=482
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/110/1054110/0e76b9541118d347871c0e0e51d244c4/8b5648f3641b3825d37695724278e6e7/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/110/1054110/0e76b9541118d347871c0e0e51d244c4/f3f93116383f70c14524df5741e87d77/ORIGINAL.jpeg
load img from url https://d3ls91xgksobn.cloudfront.net:443/651x434,fit/etuovimedia/images/property/import/110/1054110/0e76b9541118d347871c0